In [3]:
!pip install pydub

In [32]:
import numpy as np
from scipy.io import wavfile
import IPython
import random

(16000, 16000, (13036,), 1)

In [82]:
def to_target_size(arr, target_size, max_crop):
    """
    This random pads left and right, but, it will also cut down files to target_size if they are bigger. 
    you can specify the max_crop, otherwise it errs.  we can do that silently if we want, just throw those away.  whatever we like. 
    returns a numpy array.  
    """
    if len(arr.shape) != 1:
        raise ValueError("You must pass in a 1-d array.")
        
    current_size = len(arr)
    
    if current_size < target_size:
        left_pad = random.randint(0,target_size - current_size)
        right_pad = target_size - current_size - left_pad
        padded_arr = np.pad(arr, (left_pad, right_pad), mode='constant')
    else:
        if current_size - max_crop > target_size:
            raise ValueError(f"Input file exceeds target_size by more than {max_crop}. ")
        crop = min(current_size - target_size, max_crop)
        left_crop = crop // 2
        right_crop = crop - left_crop

        # Crop the array
        padded_arr = arr[left_crop:(current_size - right_crop)]

    return padded_arr

In [142]:
def generate_white_noise(duration):
    #-32768 to 32767 noise range for audio 
    # per https://stackoverflow.com/questions/5890499/pcm-audio-amplitude-values
    noise = np.random.randint(low=-32768,high=32767,size=(duration),dtype=int)
    return noise

def add_white_noise(arr,max_noise=0.1):
    #randomly sets the volume of the noise between 0 and max_noise
    #mixes and returns array
    #0.1 is like, lots of noise fyi.  intelligible but, very there.
    array_len = len(arr)
    noise = generate_white_noise(array_len) #make noise the same size as the input array
    noise_volume = random.random()*max_noise
    volume_arr = 1 - noise_volume
    
    #print(noise_volume)
    merged_audio = (arr * volume_arr + noise * noise_volume) 
    return merged_audio, noise_volume

    

In [143]:
rate1, audio1 = wavfile.read("data/down/en-US_48Z9LUDWGL9J8KH.wav")
rate2, audio2 = wavfile.read("data/up/en-US_3BV6T87TTZVNQ70.wav")
print(rate1,rate2)#These should be 16k maybe do some asserting?
audio1.shape,len(audio2.shape) 

16000 16000


((13036,), 1)

In [144]:
target_size = 16_000
max_crop = 100 #this isn't important for this but, yeah, when we loop we may wanna take it into consideration
a1_16k = to_target_size(audio1, target_size, max_crop)
a2_16k = to_target_size(audio2, target_size, max_crop)

a1_noised,_ = add_white_noise(a1_16k,max_noise=0.1)
a2_noised,_ = add_white_noise(a2_16k,max_noise=0.1)
len(a1_noised),len(a2_noised)

(16000, 16000)

In [145]:
IPython.display.Audio(a1_noised,rate=16_000)

In [146]:
IPython.display.Audio(a2_noised,rate=16_000)